In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from datetime import datetime
import re

### 2. Configuration et fonctions d'extraction

In [2]:
def setup_driver():
    """
    Configure et initialise le driver Selenium
    """
    chrome_options = Options()
    # Commenter la ligne suivante pour voir le navigateur (utile pour debug)
    chrome_options.add_argument('--headless')  
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    chrome_options.add_argument('--window-size=1920,1080')  # Taille de fenêtre pour voir tous les éléments
    chrome_options.add_argument('--disable-gpu')
    
    # Options pour éviter la détection
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Masquer l'indicateur webdriver
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    driver.set_page_load_timeout(30)
    
    return driver

### 3. Fonctions d'extraction des données

In [3]:
def wait_for_element(driver, by, value, timeout=10):
    """
    Attend qu'un élément soit présent sur la page
    """
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((by, value))
        )
        return element
    except:
        return None

In [4]:
def extract_text_safe(driver, selector, by=By.CSS_SELECTOR):
    """
    Extrait le texte d'un élément de manière sécurisée
    """
    try:
        element = driver.find_element(by, selector)
        return element.text.strip() if element else 'N/A'
    except:
        return 'N/A'


In [5]:
def extract_car_details_selenium(driver, url):
    """
    Extrait tous les détails d'une voiture avec Selenium
    """
    try:
        print(f"  📄 Chargement de la page...")
        driver.get(url)
        
        # Attendre que le contenu principal soit chargé
        time.sleep(3)  # Laisser le temps au JavaScript de s'exécuter
        
        # Attendre l'élément principal
        wait_for_element(driver, By.CLASS_NAME, "product-name", timeout=15)
        
        car_data = {}
        
        # Extraction du nom du produit
        car_data['Nom'] = extract_text_safe(driver, '.product-name')
        
        # Extraction de la description
        car_data['Description'] = extract_text_safe(driver, '.product-description')
        
        # Extraction du prix
        try:
            price_elem = driver.find_element(By.CLASS_NAME, 'price')
            price_text = price_elem.text.strip()
            # Nettoyer le prix
            price_clean = re.sub(r'[^\d]', '', price_text)
            car_data['Prix'] = price_clean if price_clean else 'N/A'
        except:
            car_data['Prix'] = 'N/A'
        
        # Extraction de la référence
        try:
            ref_elem = driver.find_element(By.CLASS_NAME, 'reference')
            ref_text = ref_elem.text.strip()
            car_data['Référence'] = ref_text.replace('Référence :', '').replace('Référence', '').strip()
        except:
            car_data['Référence'] = 'N/A'
        
        # Extraction de la main
        try:
            main_elem = driver.find_element(By.CSS_SELECTOR, '.kilometrage.badge')
            car_data['Main'] = main_elem.text.strip()
        except:
            car_data['Main'] = 'N/A'
        
        # Extraction des features (caractéristiques)
        try:
            features = driver.find_elements(By.CSS_SELECTOR, '.features .feature')
            
            for feature in features:
                try:
                    title_elem = feature.find_element(By.CLASS_NAME, 'title')
                    content_elem = feature.find_element(By.CLASS_NAME, 'content')
                    
                    title = title_elem.text.strip()
                    content = content_elem.text.strip()
                    
                    if title and content:
                        car_data[title] = content
                except:
                    continue
        except Exception as e:
            print(f"  ⚠️ Erreur lors de l'extraction des features: {e}")
        
        # URL de la voiture
        car_data['URL'] = url
        
        print(f"  ✓ Données extraites: {car_data.get('Nom', 'Inconnu')}")
        return car_data
        
    except Exception as e:
        print(f"  ❌ Erreur lors de l'extraction: {e}")
        return None


In [6]:
def extract_car_details_selenium(driver, url):
    """
    Extrait tous les détails d'une voiture avec Selenium
    Version améliorée avec sélecteurs précis basés sur le HTML réel
    """
    try:
        print(f"  📄 Chargement de la page...")
        driver.get(url)
        
        # Attendre plus longtemps pour le chargement JavaScript
        time.sleep(7)
        
        # Scroll pour déclencher le lazy loading
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
        time.sleep(2)
        
        car_data = {}
        
        # ==================== NOM DU PRODUIT ====================
        print("  🔍 Extraction du nom...")
        try:
            # Le nom est dans <span class="product-name text-black fw-bolder fs-2 mb-0 ms-2">
            name_elem = driver.find_element(By.CSS_SELECTOR, 'span.product-name')
            car_data['Nom'] = name_elem.text.strip()
            print(f"    ✓ Nom: {car_data['Nom']}")
        except Exception as e:
            print(f"    ❌ Erreur nom: {e}")
            car_data['Nom'] = 'N/A'
        
        # ==================== DESCRIPTION ====================
        print("  🔍 Extraction de la description...")
        try:
            # <p class="product-description fw-meduim ">AMG LINE </p>
            desc_elem = driver.find_element(By.CSS_SELECTOR, 'p.product-description')
            car_data['Description'] = desc_elem.text.strip()
            print(f"    ✓ Description: {car_data['Description']}")
        except Exception as e:
            print(f"    ❌ Erreur description: {e}")
            car_data['Description'] = 'N/A'
        
        # ==================== PRIX ====================
        print("  🔍 Extraction du prix...")
        try:
            # Structure: <p class="price mb-0 p-0"><span class="mb-0">145</span>,000 DT</p>
            price_elem = driver.find_element(By.CSS_SELECTOR, 'p.price')
            price_full_text = price_elem.text.strip()
            
            # Alternative: essayer de trouver directement le span avec le nombre
            try:
                price_span = price_elem.find_element(By.TAG_NAME, 'span')
                price_number = price_span.text.strip()
                # Récupérer le reste du texte après le span
                remaining_text = price_elem.text.replace(price_number, '').strip()
                car_data['Prix'] = f"{price_number}{remaining_text}"
            except:
                car_data['Prix'] = price_full_text
            
            print(f"    ✓ Prix: {car_data['Prix']}")
        except Exception as e:
            print(f"    ❌ Erreur prix: {e}")
            car_data['Prix'] = 'N/A'
        
        # ==================== RÉFÉRENCE ====================
        print("  🔍 Extraction de la référence...")
        try:
            # <span class="reference text-black">Référence : 6208</span>
            ref_elem = driver.find_element(By.CSS_SELECTOR, 'span.reference')
            ref_text = ref_elem.text.strip()
            
            # Extraire juste le numéro
            if ':' in ref_text:
                car_data['Référence'] = ref_text.split(':')[1].strip()
            else:
                car_data['Référence'] = ref_text.replace('Référence', '').strip()
            
            print(f"    ✓ Référence: {car_data['Référence']}")
        except Exception as e:
            print(f"    ❌ Erreur référence: {e}")
            car_data['Référence'] = 'N/A'
        
        # ==================== MAIN ====================
        print("  🔍 Extraction de la main...")
        try:
            # <span class="kilometrage badge fw-medium"><span class="fw-bold">1 ere Main </span></span>
            # Essayer plusieurs sélecteurs
            selectors = [
                'span.kilometrage.badge',
                'span.badge.kilometrage',
                '.kilometrage.badge span.fw-bold'
            ]
            
            main_found = False
            for selector in selectors:
                try:
                    main_elem = driver.find_element(By.CSS_SELECTOR, selector)
                    car_data['Main'] = main_elem.text.strip()
                    main_found = True
                    print(f"    ✓ Main: {car_data['Main']}")
                    break
                except:
                    continue
            
            if not main_found:
                car_data['Main'] = 'N/A'
                print(f"    ⚠️ Main non trouvée")
                
        except Exception as e:
            print(f"    ❌ Erreur main: {e}")
            car_data['Main'] = 'N/A'
        
        # ==================== FEATURES (Caractéristiques) ====================
        print("  🔍 Extraction des features...")
        try:
            # Attendre que le container features soit chargé
            features_container = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'features-container'))
            )
            
            # Trouver toutes les features dans <div class="features">
            features = driver.find_elements(By.CSS_SELECTOR, '.features .feature')
            
            print(f"    ℹ️ Nombre de features trouvées: {len(features)}")
            
            feature_count = 0
            for feature in features:
                try:
                    # Structure: 
                    # <div class="feature">
                    #   <span class="title">Kilométrage</span>
                    #   <span class="content">3 200 Km</span>
                    # </div>
                    
                    title_elem = feature.find_element(By.CSS_SELECTOR, 'span.title')
                    content_elem = feature.find_element(By.CSS_SELECTOR, 'span.content')
                    
                    title = title_elem.text.strip()
                    content = content_elem.text.strip()
                    
                    if title and content:
                        car_data[title] = content
                        feature_count += 1
                        print(f"    ✓ {title}: {content}")
                        
                except Exception as e:
                    continue
            
            print(f"    ✓ Total features extraites: {feature_count}")
            
        except Exception as e:
            print(f"    ⚠️ Erreur lors de l'extraction des features: {e}")
        
        # ==================== URL ====================
        car_data['URL'] = url
        
        # ==================== RÉSUMÉ ====================
        print(f"\n  ✅ EXTRACTION TERMINÉE")
        print(f"    - Nom: {car_data.get('Nom', 'N/A')}")
        print(f"    - Prix: {car_data.get('Prix', 'N/A')}")
        print(f"    - Référence: {car_data.get('Référence', 'N/A')}")
        print(f"    - Main: {car_data.get('Main', 'N/A')}")
        print(f"    - Caractéristiques: {len([k for k in car_data.keys() if k not in ['Nom', 'Description', 'Prix', 'Référence', 'Main', 'URL']])}")
        
        return car_data
        
    except Exception as e:
        print(f"  ❌ Erreur générale lors de l'extraction: {e}")
        import traceback
        traceback.print_exc()
        return None

### 4. Fonction pour récupérer les liens des voitures

In [7]:
def get_car_links_from_all_pages(driver, base_url, max_pages=22, start_page=1, max_cars=None):
    """
    Récupère les liens des voitures depuis toutes les pages avec pagination AJAX
    max_pages: nombre de pages à scraper
    start_page: page de départ (par défaut 1)
    max_cars: nombre maximum de voitures à récupérer (None = toutes)
    """
    all_car_links = []
    
    try:
        end_page = start_page + max_pages - 1
        print(f"🔍 Récupération des liens depuis les pages {start_page} à {end_page}...")
        if max_cars:
            print(f"   Limite: {max_cars} voitures maximum")
        
        # Charger la première page
        print(f"\n📄 Chargement initial de la page...")
        driver.get(base_url)
        time.sleep(5)
        
        # Si start_page > 1, naviguer jusqu'à cette page d'abord
        if start_page > 1:
            print(f"\n🔄 Navigation vers la page de départ ({start_page})...")
            for page in range(2, start_page + 1):
                try:
                    page_button = driver.find_element(
                        By.XPATH, 
                        f"//ul[@class='pagination']//a[contains(text(), '{page}') and contains(@onclick, 'loadAjaxListing')]"
                    )
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", page_button)
                    time.sleep(1)
                    page_button.click()
                    time.sleep(5)
                    print(f"  ✓ Page {page} chargée")
                except Exception as e:
                    print(f"  ❌ Erreur navigation page {page}: {e}")
        
        for page_num in range(start_page, end_page + 1):
            # Vérifier si on a atteint la limite de voitures
            if max_cars and len(all_car_links) >= max_cars:
                print(f"\n✅ Limite de {max_cars} voitures atteinte")
                break
            
            print(f"\n📄 Page {page_num}")
            
            # Si ce n'est pas la page de départ, cliquer sur le bouton de pagination
            if page_num > start_page:
                try:
                    # Attendre que la pagination soit visible
                    time.sleep(2)
                    
                    # Chercher le bouton de la page dans la pagination
                    # Le bouton contient le numéro de page et un onclick avec loadAjaxListing
                    page_button = driver.find_element(
                        By.XPATH, 
                        f"//ul[@class='pagination']//a[contains(text(), '{page_num}') and contains(@onclick, 'loadAjaxListing')]"
                    )
                    
                    print(f"  🖱️ Clic sur le bouton page {page_num}...")
                    
                    # Scroll jusqu'au bouton pour le rendre visible
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", page_button)
                    time.sleep(1)
                    
                    # Cliquer sur le bouton
                    page_button.click()
                    
                    # Attendre le chargement AJAX
                    print(f"  ⏳ Attente du chargement AJAX...")
                    time.sleep(5)
                    
                    # Vérifier que la page active a changé
                    try:
                        active_page = driver.find_element(By.CSS_SELECTOR, '.pagination .page-item.active .page-link')
                        print(f"  ✓ Page active: {active_page.text}")
                    except:
                        print(f"  ⚠️ Impossible de vérifier la page active")
                    
                except Exception as e:
                    print(f"  ❌ Erreur lors du clic sur la page {page_num}: {e}")
                    continue
            
            # Extraire les liens de la page actuelle
            try:
                # Trouver tous les liens de voitures
                link_elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="listing-detail"]')
                
                page_links = []
                for elem in link_elements:
                    # Vérifier si on a atteint la limite
                    if max_cars and len(all_car_links) >= max_cars:
                        break
                        
                    href = elem.get_attribute('href')
                    if href and 'listing-detail' in href and href not in all_car_links:
                        page_links.append(href)
                        all_car_links.append(href)
                
                print(f"  ✓ {len(page_links)} voiture(s) trouvée(s) sur cette page")
                print(f"  📊 Total actuel: {len(all_car_links)} voitures")
                
            except Exception as e:
                print(f"  ❌ Erreur lors de l'extraction des liens: {e}")
                continue
        
        # Supprimer les doublons
        all_car_links = list(dict.fromkeys(all_car_links))
        
        print(f"\n✅ Total: {len(all_car_links)} lien(s) unique(s) trouvé(s)")
        return all_car_links
        
    except Exception as e:
        print(f"❌ Erreur générale: {e}")
        import traceback
        traceback.print_exc()
        return all_car_links


### 5. Fonction principale de scraping

In [8]:
def scrape_spark_auto_selenium(max_pages=22, start_page=1, max_cars=None):
    """
    Fonction principale pour scraper les voitures avec Selenium
    max_pages: nombre de pages à scraper
    start_page: page de départ (par défaut 1)
    max_cars: nombre maximum de voitures à scraper (None = toutes)
    """
    print(f"🚀 Début du scraping avec Selenium - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Pages à scraper: {start_page} à {start_page + max_pages - 1}")
    if max_cars:
        print(f"Limite: {max_cars} voitures maximum")
    print()
    
    driver = None
    cars_data = []
    
    try:
        # Initialiser le driver
        print("🔧 Configuration du WebDriver...")
        driver = setup_driver()
        print("✓ WebDriver configuré\n")
        
        base_url = "https://www.sparkauto.tn/achat-voiture-occasion-tunisie"
        
        # Récupérer les liens de toutes les voitures
        car_urls = get_car_links_from_all_pages(driver, base_url, max_pages, start_page, max_cars)
        
        if not car_urls:
            print("❌ Aucun lien de voiture trouvé")
            return pd.DataFrame()
        
        # Extraire les détails de chaque voiture
        for i, url in enumerate(car_urls, 1):
            print(f"\n[{i}/{len(car_urls)}] Traitement: {url}")
            
            car_details = extract_car_details_selenium(driver, url)
            
            if car_details:
                cars_data.append(car_details)
                print(f"  ✅ Voiture ajoutée")
            else:
                print(f"  ❌ Échec de l'extraction")
            
            # Pause entre les requêtes
            if i < len(car_urls):
                time.sleep(2)
        
    except Exception as e:
        print(f"\n❌ Erreur générale: {e}")
    
    finally:
        # Fermer le driver
        if driver:
            driver.quit()
            print("\n🔒 WebDriver fermé")
    
    # Créer le DataFrame
    if cars_data:
        df = pd.DataFrame(cars_data)
        print(f"\n✅ Scraping terminé! {len(cars_data)} voiture(s) extraite(s)")
        return df
    else:
        print("\n❌ Aucune donnée extraite")
        return pd.DataFrame()

### 6. Exécution du scraping

In [39]:
# Scraper 20 voitures des pages 4 et 5
df_voitures = scrape_spark_auto_selenium(max_pages=4, start_page=18, max_cars=200)

🚀 Début du scraping avec Selenium - 2025-11-28 11:59:44
Pages à scraper: 18 à 21
Limite: 200 voitures maximum

🔧 Configuration du WebDriver...
✓ WebDriver configuré

🔍 Récupération des liens depuis les pages 18 à 21...
   Limite: 200 voitures maximum

📄 Chargement initial de la page...
✓ WebDriver configuré

🔍 Récupération des liens depuis les pages 18 à 21...
   Limite: 200 voitures maximum

📄 Chargement initial de la page...

🔄 Navigation vers la page de départ (18)...

🔄 Navigation vers la page de départ (18)...
  ✓ Page 2 chargée
  ✓ Page 2 chargée
  ✓ Page 3 chargée
  ✓ Page 3 chargée
  ✓ Page 4 chargée
  ✓ Page 4 chargée
  ✓ Page 5 chargée
  ✓ Page 5 chargée
  ✓ Page 6 chargée
  ✓ Page 6 chargée
  ✓ Page 7 chargée
  ✓ Page 7 chargée
  ✓ Page 8 chargée
  ✓ Page 8 chargée
  ✓ Page 9 chargée
  ✓ Page 9 chargée
  ✓ Page 10 chargée
  ✓ Page 10 chargée
  ✓ Page 11 chargée
  ✓ Page 11 chargée
  ✓ Page 12 chargée
  ✓ Page 12 chargée
  ✓ Page 13 chargée
  ✓ Page 13 chargée
  ✓ Page 14 cha

### 7. Affichage des résultats

In [40]:
# Afficher les résultats
if not df_voitures.empty:
    print("\n" + "="*100)
    print("RÉSULTATS DU SCRAPING")
    print("="*100)
    display(df_voitures)
    
    print("\n" + "="*100)
    print("INFORMATIONS SUR LE DATASET")
    print("="*100)
    print(f"Nombre de voitures: {len(df_voitures)}")
    print(f"Nombre de colonnes: {len(df_voitures.columns)}")
    print(f"\nColonnes extraites:")
    for col in df_voitures.columns:
        print(f"  • {col}")



RÉSULTATS DU SCRAPING


,Nom,Description,Prix,Référence,Main,Kilométrage,Catégorie,1ère Immat.,Puissance Fiscale,Moteur,Carburant,Consommation,Transmission,URL
0,BMW Série 5,520i,"68,000 DT",1888,2eme main,172 000 Km,Berline,Juillet 2014,10 cv,184 cv,Essence,"6,1 L/100 Km",Automatique,https://www.sparkauto.tn/listing-detail/138/BM...
1,Peugeot 3008,1.2 Puretech,"69,000 DT",1794,1ere main,93 000 Km,SUV,Novembre 2020,7 cv,130 cv,Essence,4.5 L / 100 km,Automatique,https://www.sparkauto.tn/listing-detail/136/Pe...
2,Volkswagen Passat,1.4 l R-line,"87,000 DT",1774,1ere main,101 000 Km,Berline,Octobre 2019,8 cv,150 cv,Essence,5.4 L / 100 km,Automatique,https://www.sparkauto.tn/listing-detail/134/Vo...
3,Kia Picanto,1.2 Gt Line,"36,500 DT",1702,1ère main,151 000 Km,Citadine,Mai 2018,4 cv,67 cv,Essence,4.9 L / 100 km,Manuelle,https://www.sparkauto.tn/listing-detail/129/Ki...
4,Audi A3,1.4 l TFSI,"104,000 DT",1667,1ere main,56 000 Km,Berline,Juillet 2022,8 cv,150 cv,Essence,4.6 L / 100 km,Automatique,https://www.sparkauto.tn/listing-detail/127/Au...
5,Fiat Punto,1.2 l,"28,000 DT",1666,1ere main,91 000 Km,Compact,Avril 2016,4 cv,69 cv,Essence,5.4 L / 100 km,Manuelle,https://www.sparkauto.tn/listing-detail/126/Fi...
6,Kia Rio,1.2 l,"57,000 DT",1633,1ere main,40 500 Km,Compact,Juillet 2020,5 cv,84 cv,Essence,5.4 L / 100 km,Manuelle,https://www.sparkauto.tn/listing-detail/123/Ki...
7,Seat Tarraco,1.4 l TSI,"107,000 DT",1480,1ere main,96 000 Km,SUV,Aout 2021,9 cv,150 cv,Essence,6.7L/100KM,Automatique,https://www.sparkauto.tn/listing-detail/121/Se...
8,Citroën C3,1.2 Puretech,"34,500 DT",1342,1ere main,67 000 Km,Compact,Octobre 2017,4 cv,82 cv,Essence,4.3 L / 100 km,Manuelle,https://www.sparkauto.tn/listing-detail/113/Ci...
9,Hyundai Tucson,1.6 l HTRACK,"125,000 DT",1301,1ere main,11 500 Km,SUV,Mai 2023,10 cv,180 ch,Essence,6.3L/100KM,Automatique,https://www.sparkauto.tn/listing-detail/108/Hy...



INFORMATIONS SUR LE DATASET
Nombre de voitures: 34
Nombre de colonnes: 14

Colonnes extraites:
  • Nom
  • Description
  • Prix
  • Référence
  • Main
  • Kilométrage
  • Catégorie
  • 1ère Immat.
  • Puissance Fiscale
  • Moteur
  • Carburant
  • Consommation
  • Transmission
  • URL


In [42]:
if not df_voitures.empty:
    print("\n" + "="*100)
    print("DÉTAILS DE LA PREMIÈRE VOITURE")
    print("="*100)
    for col, value in df_voitures.iloc[9].items():
        print(f"{col:25s}: {value}")


DÉTAILS DE LA PREMIÈRE VOITURE
Nom                      : Hyundai Tucson
Description              : 1.6 l HTRACK
Prix                     : 125,000 DT
Référence                : 1301
Main                     : 1ere main
Kilométrage              : 11 500 Km
Catégorie                : SUV
1ère Immat.              : Mai 2023
Puissance Fiscale        : 10 cv
Moteur                   : 180 ch
Carburant                : Essence
Consommation             : 6.3L/100KM
Transmission             : Automatique
URL                      : https://www.sparkauto.tn/listing-detail/108/HyundaiTucson


### 8. Nettoyage et transformation des données

In [43]:
def clean_dataframe(df):
    """
    Nettoie et transforme les données
    """
    if df.empty:
        return df
    
    df_clean = df.copy()
    
    # Nettoyer le prix
    if 'Prix' in df_clean.columns:
        df_clean['Prix_Numeric'] = pd.to_numeric(
            df_clean['Prix'].astype(str).str.replace(' ', '').str.replace(',', '').str.replace('DT', ''),
            errors='coerce'
        )
    
    # Nettoyer le kilométrage
    if 'Kilométrage' in df_clean.columns:
        df_clean['Kilométrage_Numeric'] = pd.to_numeric(
            df_clean['Kilométrage'].astype(str).str.replace(' ', '').str.replace('Km', '').str.replace('km', ''),
            errors='coerce'
        )
    
    # Extraire l'année de la 1ère immatriculation
    if '1ère Immat.' in df_clean.columns:
        df_clean['Année'] = df_clean['1ère Immat.'].astype(str).str.extract(r'(\d{4})', expand=False)
    
    return df_clean

In [44]:
# Nettoyer les données
if not df_voitures.empty:
    df_voitures_clean = clean_dataframe(df_voitures)
    print("\n✓ Données nettoyées")


✓ Données nettoyées


In [45]:
df_voitures_clean


,Nom,Description,Prix,Référence,Main,Kilométrage,Catégorie,1ère Immat.,Puissance Fiscale,Moteur,Carburant,Consommation,Transmission,URL,Prix_Numeric,Kilométrage_Numeric,Année
0,BMW Série 5,520i,"68,000 DT",1888,2eme main,172 000 Km,Berline,Juillet 2014,10 cv,184 cv,Essence,"6,1 L/100 Km",Automatique,https://www.sparkauto.tn/listing-detail/138/BM...,68000,172000,2014
1,Peugeot 3008,1.2 Puretech,"69,000 DT",1794,1ere main,93 000 Km,SUV,Novembre 2020,7 cv,130 cv,Essence,4.5 L / 100 km,Automatique,https://www.sparkauto.tn/listing-detail/136/Pe...,69000,93000,2020
2,Volkswagen Passat,1.4 l R-line,"87,000 DT",1774,1ere main,101 000 Km,Berline,Octobre 2019,8 cv,150 cv,Essence,5.4 L / 100 km,Automatique,https://www.sparkauto.tn/listing-detail/134/Vo...,87000,101000,2019
3,Kia Picanto,1.2 Gt Line,"36,500 DT",1702,1ère main,151 000 Km,Citadine,Mai 2018,4 cv,67 cv,Essence,4.9 L / 100 km,Manuelle,https://www.sparkauto.tn/listing-detail/129/Ki...,36500,151000,2018
4,Audi A3,1.4 l TFSI,"104,000 DT",1667,1ere main,56 000 Km,Berline,Juillet 2022,8 cv,150 cv,Essence,4.6 L / 100 km,Automatique,https://www.sparkauto.tn/listing-detail/127/Au...,104000,56000,2022
5,Fiat Punto,1.2 l,"28,000 DT",1666,1ere main,91 000 Km,Compact,Avril 2016,4 cv,69 cv,Essence,5.4 L / 100 km,Manuelle,https://www.sparkauto.tn/listing-detail/126/Fi...,28000,91000,2016
6,Kia Rio,1.2 l,"57,000 DT",1633,1ere main,40 500 Km,Compact,Juillet 2020,5 cv,84 cv,Essence,5.4 L / 100 km,Manuelle,https://www.sparkauto.tn/listing-detail/123/Ki...,57000,40500,2020
7,Seat Tarraco,1.4 l TSI,"107,000 DT",1480,1ere main,96 000 Km,SUV,Aout 2021,9 cv,150 cv,Essence,6.7L/100KM,Automatique,https://www.sparkauto.tn/listing-detail/121/Se...,107000,96000,2021
8,Citroën C3,1.2 Puretech,"34,500 DT",1342,1ere main,67 000 Km,Compact,Octobre 2017,4 cv,82 cv,Essence,4.3 L / 100 km,Manuelle,https://www.sparkauto.tn/listing-detail/113/Ci...,34500,67000,2017
9,Hyundai Tucson,1.6 l HTRACK,"125,000 DT",1301,1ere main,11 500 Km,SUV,Mai 2023,10 cv,180 ch,Essence,6.3L/100KM,Automatique,https://www.sparkauto.tn/listing-detail/108/Hy...,125000,11500,2023


### 9. Statistiques descriptives

In [46]:
# Statistiques
if not df_voitures.empty:
    df_clean = clean_dataframe(df_voitures)
    
    print("\n" + "="*100)
    print("STATISTIQUES DESCRIPTIVES")
    print("="*100)
    
    if 'Prix_Numeric' in df_clean.columns:
        print("\n📊 STATISTIQUES SUR LES PRIX:")
        print(f"  Prix moyen:    {df_clean['Prix_Numeric'].mean():,.0f} DT")
        print(f"  Prix minimum:  {df_clean['Prix_Numeric'].min():,.0f} DT")
        print(f"  Prix maximum:  {df_clean['Prix_Numeric'].max():,.0f} DT")
        print(f"  Prix médian:   {df_clean['Prix_Numeric'].median():,.0f} DT")
    
    if 'Kilométrage_Numeric' in df_clean.columns:
        print("\n🚗 STATISTIQUES SUR LE KILOMÉTRAGE:")
        print(f"  KM moyen:      {df_clean['Kilométrage_Numeric'].mean():,.0f} Km")
        print(f"  KM minimum:    {df_clean['Kilométrage_Numeric'].min():,.0f} Km")
        print(f"  KM maximum:    {df_clean['Kilométrage_Numeric'].max():,.0f} Km")


STATISTIQUES DESCRIPTIVES

📊 STATISTIQUES SUR LES PRIX:
  Prix moyen:    78,947 DT
  Prix minimum:  28,000 DT
  Prix maximum:  285,000 DT
  Prix médian:   68,500 DT

🚗 STATISTIQUES SUR LE KILOMÉTRAGE:
  KM moyen:      90,221 Km
  KM minimum:    11,500 Km
  KM maximum:    172,000 Km


### 10. Sauvegarde des données

In [47]:
if not df_voitures.empty:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # CSV
    csv_filename = f"spark_auto_voitures_{timestamp}5.csv"
    df_voitures.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"\n💾 Données sauvegardées dans: {csv_filename}")
    
    # Excel
    try:
        excel_filename = f"spark_auto_voitures_{timestamp}5.xlsx"
        df_voitures.to_excel(excel_filename, index=False, engine='openpyxl')
        print(f"💾 Données sauvegardées dans: {excel_filename}")
    except:
        print("⚠️ Installation d'openpyxl nécessaire pour Excel")


💾 Données sauvegardées dans: spark_auto_voitures_20251128_1216235.csv
💾 Données sauvegardées dans: spark_auto_voitures_20251128_1216235.xlsx
